# Assignment week 3 - part 3 - neighbourhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from urllib.request import urlopen
from bs4 import BeautifulSoup

## Part 1 (as previously submitted)

Get HTML and create BeautifulSoup object. Find the table within the page and strip the strings therein.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'html')

In [3]:
# soup.table.find_all('td')
raw_postcodes = []
for string in soup.table.stripped_strings:
    raw_postcodes.append(string)

Create a dataframe with the right column names.

In [4]:
column_names = ['PostalCode','Borough','Neighborhood']
df_postcodes = pd.DataFrame(columns=column_names)

Fill each column with the key data from the scraped website. Remove rows where Borough is not assigned.

In [5]:
df_postcodes['PostalCode'] = raw_postcodes[3::3]
df_postcodes['Borough'] = raw_postcodes[4::3]
df_postcodes['Neighborhood'] = raw_postcodes[5::3]
df_postcodes = df_postcodes[df_postcodes.Borough != 'Not assigned']

Combine rows where postcodes are repeated. Aggregate data in the Neighborhood column. Replace any Neighborhoods which show as "Not Assigned" with the corresponding Borough name.

In [6]:
grouped_postcodes = df_postcodes.groupby(['PostalCode','Borough'], sort=False).agg(lambda x: ', '.join(x)).reset_index()
grouped_postcodes.loc[grouped_postcodes.Neighborhood == 'Not assigned', 'Neighborhood'] = grouped_postcodes.Borough
grouped_postcodes.head(10)

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Harbourfront, Regent Park
3        M6A        North York  Lawrence Heights, Lawrence Manor
4        M7A      Queen's Park                      Queen's Park
5        M9A         Etobicoke                  Islington Avenue
6        M1B       Scarborough                    Rouge, Malvern
7        M3B        North York                   Don Mills North
8        M4B         East York   Woodbine Gardens, Parkview Hill
9        M5B  Downtown Toronto          Ryerson, Garden District

In [7]:
grouped_postcodes.shape

(103, 3)

## Part 2 (as previously submitted)

Getting the latitude and the longitude coordinates of each neighborhood - and creating a new dataframe with combined data.

In [8]:
# import geocoder

Couldn't reliably get geocoder to work - using csv file instead (per instructions).

In [9]:
coords = pd.read_csv('Geospatial_Coordinates.csv')
coords.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [10]:
grouped_postcodes_ll = grouped_postcodes.merge(coords, how='left', left_on='PostalCode', right_on='Postal Code')
grouped_postcodes_ll.drop(columns=['Postal Code'], inplace = True)
grouped_postcodes_ll.head(10)

PostalCode           Borough                      Neighborhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Harbourfront, Regent Park  43.654260   
3        M6A        North York  Lawrence Heights, Lawrence Manor  43.718518   
4        M7A      Queen's Park                      Queen's Park  43.662301   
5        M9A         Etobicoke                  Islington Avenue  43.667856   
6        M1B       Scarborough                    Rouge, Malvern  43.806686   
7        M3B        North York                   Don Mills North  43.745906   
8        M4B         East York   Woodbine Gardens, Parkview Hill  43.706397   
9        M5B  Downtown Toronto          Ryerson, Garden District  43.657162   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494  
5 -79.532242  
6 -79.194353  
7 -79.352188  
8 -79.309937  
9 -79.378937

## Part 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

* to add enough Markdown cells to explain what you decided to do and to report any observations you make.
* to generate maps to visualize your neighborhoods and how they cluster together.

In [11]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

### Create dataframe with subset of postal codes for analysis

What's the lat and long for Toronto? Find using geopy.

In [12]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create map of Toronto showing all postcodes. Using latitude + 0.5 to ensure all postcodes show and are centered nicely.

In [13]:
map_toronto = folium.Map(location=[latitude+0.05, longitude], zoom_start=11)

# add markers to map
for lat, lng, postcode in zip(grouped_postcodes_ll['Latitude'], 
                              grouped_postcodes_ll['Longitude'], 
                              grouped_postcodes_ll['PostalCode']):
    label = '{}'.format(postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Decided to focus on boroughs that contain the word Toronto - so create new df on that basis and replot map.

In [14]:
focus_postcodes = grouped_postcodes_ll[grouped_postcodes_ll['Borough'].str.contains('Toronto')]
focus_postcodes.reset_index(inplace=True)
focus_postcodes.head()

index PostalCode           Borough               Neighborhood   Latitude  \
0      2        M5A  Downtown Toronto  Harbourfront, Regent Park  43.654260   
1      9        M5B  Downtown Toronto   Ryerson, Garden District  43.657162   
2     15        M5C  Downtown Toronto             St. James Town  43.651494   
3     19        M4E      East Toronto                The Beaches  43.676357   
4     20        M5E  Downtown Toronto                Berczy Park  43.644771   

   Longitude  
0 -79.360636  
1 -79.378937  
2 -79.375418  
3 -79.293031  
4 -79.373306

In [15]:
focus_postcodes.shape

(38, 6)

In [16]:
map_toronto = folium.Map(location=[latitude+0.02, longitude], zoom_start=12)

# add markers to map
for lat, lng, postcode in zip(focus_postcodes['Latitude'], 
                              focus_postcodes['Longitude'], 
                              focus_postcodes['PostalCode']):
    label = '{}'.format(postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Use Foursquare to find venue information for each postal code

In [40]:
# @hidden_cell

CLIENT_ID = 'REMOVED' # your Foursquare ID
CLIENT_SECRET = 'REMOVED' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: REMOVED
CLIENT_SECRET:REMOVED


In [18]:
focus_postcodes.head()

index PostalCode           Borough               Neighborhood   Latitude  \
0      2        M5A  Downtown Toronto  Harbourfront, Regent Park  43.654260   
1      9        M5B  Downtown Toronto   Ryerson, Garden District  43.657162   
2     15        M5C  Downtown Toronto             St. James Town  43.651494   
3     19        M4E      East Toronto                The Beaches  43.676357   
4     20        M5E  Downtown Toronto                Berczy Park  43.644771   

   Longitude  
0 -79.360636  
1 -79.378937  
2 -79.375418  
3 -79.293031  
4 -79.373306

Let's look at one specific postcode to explore data shaping etc.

In [19]:
pc_lat = focus_postcodes.Latitude.loc[0]
pc_long = focus_postcodes.Longitude.loc[0]
pc_name = focus_postcodes.PostalCode.loc[0]

print('Postcode of interest {}, has lat {} and long {}'.format(pc_name,pc_lat,pc_long))

Postcode of interest M5A, has lat 43.6542599 and long -79.3606359


In [20]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    pc_lat, 
    pc_long, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FEQWDLEIPOQ3CQHSC5CJYGP3AZNLMISYWU320DIRXSOHH1WL&client_secret=IX4I4V02TVPD3PIQSHLINLVGMKGOP12F1QZCP3RBNXDBIB35&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

For test postcode, get results using URL defined above. Define function to extract category of the venues and then run this on the test postcode. It looks good! 

In [21]:
results = requests.get(url).json()

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name            categories  \
0                                 Roselle Desserts                Bakery   
1                                    Tandem Coffee           Coffee Shop   
2  Toronto Cooper Koo Family Cherry St YMCA Centre  Gym / Fitness Center   
3                               Morning Glory Cafe        Breakfast Spot   
4                              Body Blitz Spa East                   Spa   

         lat        lng  
0  43.653447 -79.362017  
1  43.653559 -79.361809  
2  43.653191 -79.357947  
3  43.653947 -79.361149  
4  43.654735 -79.359874

In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


Define function to do this for the various postcodes we have decided to look at.

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run function for all Toronto postcodes of focus.

In [26]:
toronto_venues = getNearbyVenues(names=focus_postcodes['PostalCode'],
                                   latitudes=focus_postcodes['Latitude'],
                                   longitudes=focus_postcodes['Longitude']
                                  )
toronto_venues.head()

M5A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


PostalCode  PostalCode Latitude  PostalCode Longitude  \
0        M5A             43.65426            -79.360636   
1        M5A             43.65426            -79.360636   
2        M5A             43.65426            -79.360636   
3        M5A             43.65426            -79.360636   
4        M5A             43.65426            -79.360636   

                                             Venue  Venue Latitude  \
0                                 Roselle Desserts       43.653447   
1                                    Tandem Coffee       43.653559   
2  Toronto Cooper Koo Family Cherry St YMCA Centre       43.653191   
3                               Morning Glory Cafe       43.653947   
4                              Body Blitz Spa East       43.654735   

   Venue Longitude        Venue Category  
0       -79.362017                Bakery  
1       -79.361809           Coffee Shop  
2       -79.357947  Gym / Fitness Center  
3       -79.361149        Breakfast Spot  
4       -79.359874                   Spa

In [27]:
print(toronto_venues.shape)
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

(1703, 7)
There are 238 uniques categories.


In [28]:
toronto_venues.groupby('PostalCode').count()

PostalCode Latitude  PostalCode Longitude  Venue  Venue Latitude  \
PostalCode                                                                     
M4E                           5                     5      5               5   
M4K                          44                    44     44              44   
M4L                          19                    19     19              19   
M4M                          37                    37     37              37   
M4N                           3                     3      3               3   
M4P                           8                     8      8               8   
M4R                          21                    21     21              21   
M4S                          35                    35     35              35   
M4T                           4                     4      4               4   
M4V                          14                    14     14              14   
M4W                           4                     4      4               4   
M4X                          48                    48     48              48   
M4Y                          85                    85     85              85   
M5A                          48                    48     48              48   
M5B                         100                   100    100             100   
M5C                         100                   100    100             100   
M5E                          56                    56     56              56   
M5G                          84                    84     84              84   
M5H                         100                   100    100             100   
M5J                         100                   100    100             100   
M5K                         100                   100    100             100   
M5L                         100                   100    100             100   
M5N                           2                     2      2               2   
M5P                           4                     4      4               4   
M5R                          24                    24     24              24   
M5S                          34                    34     34              34   
M5T                         100                   100    100             100   
M5V                          15                    15     15              15   
M5W                          95                    95     95              95   
M5X                         100                   100    100             100   
M6G                          15                    15     15              15   
M6H                          20                    20     20              20   
M6J                          62                    62     62              62   
M6K                          21                    21     21              21   
M6P                          24                    24     24              24   
M6R                          16                    16     16              16   
M6S                          37                    37     37              37   
M7Y                          19                    19     19              19   

            Venue Longitude  Venue Category  
PostalCode                                   
M4E                       5               5  
M4K                      44              44  
M4L                      19              19  
M4M                      37              37  
M4N                       3               3  
M4P                       8               8  
M4R                      21              21  
M4S                      35              35  
M4T                       4               4  
M4V                      14              14  
M4W                       4               4  
M4X                      48              48  
M4Y                      85              85  
M5A                      48              48  
M5B                     100             100  
M5C                     100  

Set up one hot encoding for venues returned.

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print('Shape is:', toronto_onehot.shape)
toronto_onehot.head()

Shape is: (1703, 239)


PostalCode  Adult Boutique  Afghan Restaurant  Airport  Airport Food Court  \
0        M5A               0                  0        0                   0   
1        M5A               0                  0        0                   0   
2        M5A               0                  0        0                   0   
3        M5A               0                  0        0                   0   
4        M5A               0                  0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Argentinian Restaurant  \
0                    0             0         0                       0   
1                    0             0         0                       0   
2                    0             0         0                       0   
3                    0             0         0                       0   
4                    0             0         0                       0   

   Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0            0           0                    0                 0   
1            0           0                    0                 0   
2            0           0                    0                 0   
3            0           0                    0                 0   
4            0           0                    0                 0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0              0          0           0           0       1     0    0   
1              0          0           0           0       0     0    0   
2              0          0           0           0       0     0    0   
3              0          0           0           0       0     0    0   
4              0          0           0           0       0     0    0   

   Baseball Stadium  Basketball Stadium  Beach  Beer Bar  Beer Store  \
0                 0                   0      0         0           0   
1                 0                   0      0         0           0   
2                 0                   0      0         0           0   
3                 0                   0      0         0           0   
4                 0                   0      0         0           0   

   Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0       0              0          0         0   
1                   0       0              0          0         0   
2                   0       0              0          0         0   
3                   0       0              0          0         0   
4                   0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  Building  \
0                     0               0        0                0         0   
1                     0               0        0                0         0   
2                     0               0        0                0         0   
3                     0               1        0                0         0   
4                     0               0        0                0         0   

   Burger Joint  Burrito Place  Bus Line  Butcher  Café  \
0             0              0         0        0     0   
1             0              0         0        0     0   
2             0              0         0        0     0   
3             0              0         0        0     0   
4             0              0         0        0     0   

   Cajun / Creole Restaurant  Camera Store  Caribbean Restaurant  Cheese Shop  \
0                          0        

Group rows for the same postcode together and average the values for each group.

In [30]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
print('Shape is:', toronto_grouped.shape)
toronto_grouped

Shape is: (38, 239)


PostalCode  Adult Boutique  Afghan Restaurant   Airport  \
0         M4E        0.000000           0.000000  0.000000   
1         M4K        0.000000           0.000000  0.000000   
2         M4L        0.000000           0.000000  0.000000   
3         M4M        0.000000           0.000000  0.000000   
4         M4N        0.000000           0.000000  0.000000   
5         M4P        0.000000           0.000000  0.000000   
6         M4R        0.000000           0.000000  0.000000   
7         M4S        0.000000           0.000000  0.000000   
8         M4T        0.000000           0.000000  0.000000   
9         M4V        0.000000           0.000000  0.000000   
10        M4W        0.000000           0.000000  0.000000   
11        M4X        0.000000           0.000000  0.000000   
12        M4Y        0.011765           0.011765  0.000000   
13        M5A        0.000000           0.000000  0.000000   
14        M5B        0.000000           0.000000  0.000000   
15        M5C        0.000000           0.000000  0.000000   
16        M5E        0.000000           0.000000  0.000000   
17        M5G        0.000000           0.000000  0.000000   
18        M5H        0.000000           0.000000  0.000000   
19        M5J        0.000000           0.000000  0.000000   
20        M5K        0.000000           0.000000  0.000000   
21        M5L        0.000000           0.000000  0.000000   
22        M5N        0.000000           0.000000  0.000000   
23        M5P        0.000000           0.000000  0.000000   
24        M5R        0.000000           0.000000  0.000000   
25        M5S        0.000000           0.000000  0.000000   
26        M5T        0.000000           0.000000  0.000000   
27        M5V        0.000000           0.000000  0.066667   
28        M5W        0.000000           0.000000  0.000000   
29        M5X        0.000000           0.000000  0.000000   
30        M6G        0.000000           0.000000  0.000000   
31        M6H        0.000000           0.000000  0.000000   
32        M6J        0.000000           0.000000  0.000000   
33        M6K        0.000000           0.000000  0.000000   
34        M6P        0.000000           0.000000  0.000000   
35        M6R        0.000000           0.000000  0.000000   
36        M6S        0.000000           0.000000  0.000000   
37        M7Y        0.000000           0.000000  0.000000   

    Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0             0.000000      0.000000        0.000000              0.0   
1             0.000000      0.000000        0.000000              0.0   
2             0.000000      0.000000        0.000000              0.0   
3             0.000000      0.000000        0.000000              0.0   
4             0.000000      0.000000        0.000000              0.0   
5             0.000000      0.000000        0.000000              0.0   
6             0.000000      0.000000        0.000000              0.0   
7             0.000000      0.000000        0.000000              0.0   
8             0.000000      0.000000        0.000000              0.0   
9             0.000000      0.000000        0.000000              0.0   
10            0.000000      0.000000        0.000000              0.0   
11            0.000000      0.000000        0.000000              0.0   
12            0.000000      0.000000        0.000000              0.0   
13            0.000000      0.000000        0.000000              0.0   
14            0.000000      0.000000        0.000000              0.0   
15            0.000000      0.000000        0.000000              0.0   
16            0.000000      0.000000        0.000000              0.0   
17            0.000000      0.000000        0.000000              0.0   
18            0.000000      0.000000        0.000000              0.0   
19            0.000000      0.000000        0.000000              0.0   
20            0.000000      0.000000        0.00000

Define function to sort by most commonly found venues. And then create a dataframe with columns showing the 10 most common venues for each postcode.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

PostalCode 1st Most Common Venue 2nd Most Common Venue  \
0        M4E     Health Food Store                  Park   
1        M4K      Greek Restaurant           Coffee Shop   
2        M4L                  Park        Sandwich Place   
3        M4M                  Café           Coffee Shop   
4        M4N                  Park           Swim School   

  3rd Most Common Venue 4th Most Common Venue   5th Most Common Venue  \
0                   Pub           Coffee Shop            Neighborhood   
1        Ice Cream Shop    Italian Restaurant  Furniture / Home Store   
2                   Gym     Food & Drink Shop      Italian Restaurant   
3             Gastropub    Italian Restaurant                  Bakery   
4              Bus Line           Yoga Studio          Discount Store   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0     Electronics Store  Ethiopian Restaurant           Event Space   
1           Pizza Place             Bookstore               Brewery   
2               Brewery         Movie Theater        Ice Cream Shop   
3   American Restaurant           Yoga Studio     Convenience Store   
4        Farmers Market    Falafel Restaurant           Event Space   

  9th Most Common Venue 10th Most Common Venue  
0    Dim Sum Restaurant    Dumpling Restaurant  
1       Bubble Tea Shop                   Café  
2            Steakhouse                    Pub  
3               Brewery     Seafood Restaurant  
4  Ethiopian Restaurant      Electronics Store

### Clustering of postal codes using datasets created

Finally, use k-means clustering to group the postcodes.

In [33]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([8, 0, 9, 0, 2, 9, 7, 0, 5, 7, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7],
      dtype=int32)

In [34]:
neighborhoods_venues_sorted.head()

PostalCode 1st Most Common Venue 2nd Most Common Venue  \
0        M4E     Health Food Store                  Park   
1        M4K      Greek Restaurant           Coffee Shop   
2        M4L                  Park        Sandwich Place   
3        M4M                  Café           Coffee Shop   
4        M4N                  Park           Swim School   

  3rd Most Common Venue 4th Most Common Venue   5th Most Common Venue  \
0                   Pub           Coffee Shop            Neighborhood   
1        Ice Cream Shop    Italian Restaurant  Furniture / Home Store   
2                   Gym     Food & Drink Shop      Italian Restaurant   
3             Gastropub    Italian Restaurant                  Bakery   
4              Bus Line           Yoga Studio          Discount Store   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0     Electronics Store  Ethiopian Restaurant           Event Space   
1           Pizza Place             Bookstore               Brewery   
2               Brewery         Movie Theater        Ice Cream Shop   
3   American Restaurant           Yoga Studio     Convenience Store   
4        Farmers Market    Falafel Restaurant           Event Space   

  9th Most Common Venue 10th Most Common Venue  
0    Dim Sum Restaurant    Dumpling Restaurant  
1       Bubble Tea Shop                   Café  
2            Steakhouse                    Pub  
3               Brewery     Seafood Restaurant  
4  Ethiopian Restaurant      Electronics Store

In [35]:
# add clustering labels
try:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except: # for scenario where we are rerunning this and the column already exists
    neighborhoods_venues_sorted.drop(columns='Cluster Labels', inplace=True) 
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = focus_postcodes

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.head() # check the last columns!

index PostalCode           Borough               Neighborhood   Latitude  \
0      2        M5A  Downtown Toronto  Harbourfront, Regent Park  43.654260   
1      9        M5B  Downtown Toronto   Ryerson, Garden District  43.657162   
2     15        M5C  Downtown Toronto             St. James Town  43.651494   
3     19        M4E      East Toronto                The Beaches  43.676357   
4     20        M5E  Downtown Toronto                Berczy Park  43.644771   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.360636               7           Coffee Shop                   Pub   
1 -79.378937               7        Clothing Store           Coffee Shop   
2 -79.375418               7           Coffee Shop                  Café   
3 -79.293031               8     Health Food Store                  Park   
4 -79.373306               7           Coffee Shop          Cocktail Bar   

  3rd Most Common Venue 4th Most Common Venue      5th Most Common Venue  \
0                Bakery                  Park                       Café   
1                  Café        Cosmetics Shop  Middle Eastern Restaurant   
2            Restaurant                 Hotel             Cosmetics Shop   
3                   Pub           Coffee Shop               Neighborhood   
4        Farmers Market                  Café         Seafood Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0    Mexican Restaurant        Breakfast Spot               Theater   
1       Thai Restaurant  Fast Food Restaurant           Pizza Place   
2                Bakery             Gastropub          Cocktail Bar   
3     Electronics Store  Ethiopian Restaurant           Event Space   
4                   Pub           Cheese Shop            Restaurant   

  9th Most Common Venue 10th Most Common Venue  
0  Gym / Fitness Center          Historic Site  
1                 Diner                  Plaza  
2    Italian Restaurant         Breakfast Spot  
3    Dim Sum Restaurant    Dumpling Restaurant  
4                Bakery             Steakhouse

In [36]:
# create map
map_clusters = folium.Map(location=[latitude+0.02, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Review results

Looking at the results with only 5 clusters we saw that that one cluster was much larger than the others. And that three of the remainly clusters only had one member. The implication was that central Toronto is very uniform, with the outer areas being more varied. 

On this basis we decided to increase the number of clusters to allow for more granularity in the areas closer to downtown. One needs to be aware of the chances of over fitting using this method. The number of clusters were increased from five by incrementing by one and then results observed. Ten clusters was the smallest number of clusters which still produces a number of clusters with only one member, but also splits the main cluster into two larger groups.

The summary of Clusters 1 and 8 can be seen below. As would be expected, they are quite similar, with a higher level of Coffee Shops in the most common groups seen in Cluster 8. 

In [37]:
toronto_merged.groupby('Cluster Labels').count()

index  PostalCode  Borough  Neighborhood  Latitude  Longitude  \
Cluster Labels                                                                  
0                  12          12       12            12        12         12   
1                   1           1        1             1         1          1   
2                   1           1        1             1         1          1   
3                   1           1        1             1         1          1   
4                   1           1        1             1         1          1   
5                   1           1        1             1         1          1   
6                   1           1        1             1         1          1   
7                  17          17       17            17        17         17   
8                   1           1        1             1         1          1   
9                   2           2        2             2         2          2   

                1st Most Common Venue  2nd Most Common Venue  \
Cluster Labels                                                 
0                                  12                     12   
1                                   1                      1   
2                                   1                      1   
3                                   1                      1   
4                                   1                      1   
5                                   1                      1   
6                                   1                      1   
7                                  17                     17   
8                                   1                      1   
9                                   2                      2   

                3rd Most Common Venue  4th Most Common Venue  \
Cluster Labels                                                 
0                                  12                     12   
1                                   1                      1   
2                                   1                      1   
3                                   1                      1   
4                                   1                      1   
5                                   1                      1   
6                                   1                      1   
7                                  17                     17   
8                                   1                      1   
9                                   2                      2   

                5th Most Common Venue  6th Most Common Venue  \
Cluster Labels                                                 
0                                  12                     12   
1                                   1                      1   
2                                   1                      1   
3                                   1                      1   
4                                   1                      1   
5                                   1                      1   
6                                   1                      1   
7                                  17                     17   
8                                   1                      1   
9                                   2                      2   

                7th Most Common Venue  8th Most Common Venue  \
Cluster Labels                                                 
0                                  12                     12   
1                                   1                      1   
2                                   1                      1   
3                                   1                      1   
4                                   1                      1   
5                                   1                      1   
6                                   1                      1   
7                                  17                     17   
8                                   1                      1   
9                               

In [38]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

PostalCode  Longitude  Cluster Labels 1st Most Common Venue  \
6         M6G -79.422564               0         Grocery Store   
8         M6H -79.442259               0                Bakery   
10        M6J -79.419750               0                   Bar   
11        M4K -79.352188               0      Greek Restaurant   
13        M6K -79.428191               0           Coffee Shop   
16        M4M -79.340923               0                  Café   
21        M6P -79.464763               0    Mexican Restaurant   
24        M6R -79.456325               0        Breakfast Spot   
25        M4S -79.388790               0        Sandwich Place   
26        M5S -79.400049               0                  Café   
29        M5T -79.400049               0                  Café   
37        M7Y -79.321558               0    Light Rail Station   

   2nd Most Common Venue          3rd Most Common Venue  \
6                   Café                           Park   
8               Pharmacy                    Supermarket   
10           Coffee Shop               Asian Restaurant   
11           Coffee Shop                 Ice Cream Shop   
13        Breakfast Spot                           Café   
16           Coffee Shop                      Gastropub   
21                  Café                            Bar   
24             Gift Shop                   Dessert Shop   
25          Dessert Shop                    Pizza Place   
26             Bookstore                     Restaurant   
29                   Bar  Vegetarian / Vegan Restaurant   
37           Yoga Studio                  Auto Workshop   

    4th Most Common Venue   5th Most Common Venue 6th Most Common Venue  \
6      Italian Restaurant             Coffee Shop                 Diner   
8    Gym / Fitness Center                    Pool           Music Venue   
10                 Bakery             Men's Store            Restaurant   
11     Italian Restaurant  Furniture / Home Store           Pizza Place   
13            Yoga Studio           Burrito Place  Caribbean Restaurant   
16     Italian Restaurant                  Bakery   American Restaurant   
21          Grocery Store     Arts & Crafts Store  Fast Food Restaurant   
24          Movie Theater                 Dog Run                   Bar   
25     Italian Restaurant                    Café            Restaurant   
26                 Bakery     Japanese Restaurant                   Bar   
29  Vietnamese Restaurant      Mexican Restaurant           Coffee Shop   
37            Pizza Place        Recording Studio            Restaurant   

    7th Most Common Venue      8th Most Common Venue  \
6              Restaurant                  Nightclub   
8          Discount Store  Middle Eastern Restaurant   
10  Vietnamese Restaurant                       Café   
11              Bookstore                    Brewery   
13             Restaurant               Climbing Gym   
16            Yoga Studio          Convenience Store   
21              Bookstore                       Park   
24             Restaurant         Italian Restaurant   
25            Coffee Shop           Sushi Restaurant   
26               Beer Bar                 Beer Store   
29                 Bakery        Dumpling Restaurant   
37                Butcher              Burrito Place   

          9th Most Common Venue 10th Most Common Venue  
6                    Baby Store      Convenience Store  
8                          Café                Brewery  
10      New American Restaurant            Pizza Place  
11              Bubble Tea Shop                   Café  
13                    Pet Store                    Bar  
16                      Brewery     Seafood Restaurant  
21               Sandwich Place              Speakeasy  
24  Eastern European Restaurant                   Bank  
25                     Pharmacy           Gourmet Shop  
26               Sandwich Place     Chinese Restaurant  
29           Chinese Restaurant           De

In [39]:
# Cluster 8
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

PostalCode  Longitude  Cluster Labels 1st Most Common Venue  \
0         M5A -79.360636               7           Coffee Shop   
1         M5B -79.378937               7        Clothing Store   
2         M5C -79.375418               7           Coffee Shop   
4         M5E -79.373306               7           Coffee Shop   
5         M5G -79.387383               7           Coffee Shop   
7         M5H -79.384568               7           Coffee Shop   
9         M5J -79.381752               7           Coffee Shop   
12        M5K -79.381576               7           Coffee Shop   
15        M5L -79.379817               7           Coffee Shop   
22        M4R -79.405678               7        Clothing Store   
23        M5R -79.405678               7           Coffee Shop   
27        M6S -79.484450               7           Coffee Shop   
30        M4V -79.400049               7                   Pub   
33        M5W -79.374846               7           Coffee Shop   
34        M4X -79.367675               7           Coffee Shop   
35        M5X -79.382280               7           Coffee Shop   
36        M4Y -79.383160               7   Japanese Restaurant   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                    Pub                Bakery                  Park   
1            Coffee Shop                  Café        Cosmetics Shop   
2                   Café            Restaurant                 Hotel   
4           Cocktail Bar        Farmers Market                  Café   
5     Italian Restaurant                  Café          Burger Joint   
7                   Café       Thai Restaurant            Steakhouse   
9                  Hotel              Aquarium                  Café   
12                  Café                 Hotel            Restaurant   
15                  Café                 Hotel            Restaurant   
22           Coffee Shop   Sporting Goods Shop           Yoga Studio   
23        Sandwich Place                  Café           Pizza Place   
27           Pizza Place                  Café    Italian Restaurant   
30           Coffee Shop    Light Rail Station   American Restaurant   
33                  Café            Restaurant                   Pub   
34            Restaurant                  Park                  Café   
35                  Café                 Hotel            Restaurant   
36           Coffee Shop               Gay Bar      Sushi Restaurant   

        5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                        Café    Mexican Restaurant        Breakfast Spot   
1   Middle Eastern Restaurant       Thai Restaurant  Fast Food Restaurant   
2              Cosmetics Shop                Bakery             Gastropub   
4          Seafood Restaurant                   Pub           Cheese Shop   
5                         Bar        Ice Cream Shop                   Spa   
7         American Restaurant          Burger Joint      Sushi Restaurant   
9          Italian Restaurant        Scenic Lookout   Fried Chicken Joint   
12        American Restaurant         Deli / Bodega    Italian Restaurant   
15        American Restaurant    Seafood Restaurant             Gastropub   
22       Fast Food Restaurant    Chinese Restaurant   Rental Car Location   
23          Indian Restaurant        Cosmetics Shop              Pharmacy   
27                   Tea Room      Sushi Restaurant     Fish & Chips Shop   
30                 Sports Bar           Supermarket      Sushi Restaurant   
33                      Hotel          Cocktail Bar    Seafood Restaurant   
34         Italian Restaurant                Bakery                   Pub   
35         Seafood Restaurant                Bakery            Steakhouse   
36                 Restaurant                  Café             Gastropub   

   8th Most Common Venue      9th Most Common Venue    10th Most Common Venue  
0                Theater       Gym / Fitness Cente